In [14]:

from langchain.prompts.example_selector.base import BaseExampleSelector
from typing import Dict, List
import numpy as np

# 定义一个自定义示例选择器
class CustomExampleSelector(BaseExampleSelector):
    def __init__(self, examples: List[Dict[str, str]]):
        self.examples = examples

    def add_example(self, example: Dict[str, str]) -> None:
        """添加新的示例到examples列表中"""
        self.examples.append(example)

    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """从examples中随机选择指定数量的示例"""
        return np.random.choice(self.examples, size=2, replace=False)



In [15]:
# 示例列表
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
]

# 实例化 CustomExampleSelector
example_selector = CustomExampleSelector(examples)

# 添加新的示例
example_selector.add_example({"input": "windy", "output": "calm"})
print(example_selector.examples)



[{'input': 'happy', 'output': 'sad'}, {'input': 'tall', 'output': 'short'}, {'input': 'energetic', 'output': 'lethargic'}, {'input': 'sunny', 'output': 'gloomy'}, {'input': 'windy', 'output': 'calm'}]


In [16]:
from langchain.prompts import FewShotPromptTemplate
from langchain import PromptTemplate

# 定义模板
example_formatter_template = """input: {input} output: {output}\\n"""

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template=example_formatter_template,
)

# 创建 FewShotPromptTemplate
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {input}\\nOutput:",
    input_variables=["input"],
)

# 格式化提示
formatted_prompt = similar_prompt.format(input="beautiful")


In [17]:
from langchain_community.chat_models import ChatTongyi
import os
from dotenv import load_dotenv
# 创建聊天模型实例
# 加载环境变量
load_dotenv()
# 初始化聊天模型
chat = ChatTongyi(
    model='qwen-plus',
    top_p=0.9,
    temperature=0.9,
    api_key=os.getenv("DASHSCOPE_API_KEY")
)
response = chat.invoke(formatted_prompt)
print(response)

content='ugly' response_metadata={'model_name': 'qwen-plus', 'finish_reason': 'stop', 'request_id': '85584cac-0554-96f6-9986-fb9a296e82d7', 'token_usage': {'input_tokens': 49, 'output_tokens': 2, 'total_tokens': 51}} id='run-ef43898b-61c8-4fed-958a-458326f8086f-0'


In [20]:
# 导入必要的库
from langchain.prompts.example_selector.base import BaseExampleSelector
from typing import Dict, List
import numpy as np
import difflib
from langchain.prompts import FewShotPromptTemplate
from langchain import PromptTemplate
from langchain_community.chat_models import ChatTongyi
import os
from dotenv import load_dotenv

# 定义类别及其关键词
CATEGORIES = {
    "手机": ["手机", "智能手机", "iPhone", "Android", "移动"],
    "电脑": ["电脑", "笔记本", "台式机", "Mac", "Windows"],
    "服装": ["服装", "衬衫", "裤子", "连衣裙", "服饰"],
}

# 定义自定义示例选择器
class CustomExampleSelector(BaseExampleSelector):
    def __init__(self, examples: List[Dict[str, str]], categories: Dict[str, List[str]]):
        """
        初始化示例选择器。

        :param examples: 示例列表，每个示例应包含 'input', 'output' 和 'category' 字段。
        :param categories: 类别定义，每个类别对应相关的关键词列表。
        """
        self.examples = examples
        self.categories = categories

    def add_example(self, example: Dict[str, str]) -> None:
        """
        添加新的示例到examples列表中。

        :param example: 包含 'input', 'output' 和 'category' 的字典。
        """
        self.examples.append(example)

    def categorize_input(self, input_text: str) -> str:
        """
        根据输入文本识别其类别。

        :param input_text: 用户输入的文本。
        :return: 识别的类别名称，如果无法识别则返回 'general'。
        """
        input_text_lower = input_text.lower()
        for category, keywords in self.categories.items():
            for keyword in keywords:
                if keyword.lower() in input_text_lower:
                    return category
        return "general"

    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """
        根据输入变量选择相关的示例。

        :param input_variables: 包含 'input' 的字典。
        :return: 选定的示例列表。
        """
        input_text = input_variables.get("input", "")
        category = self.categorize_input(input_text)
        print(f"识别的类别: {category}")

        if category != "general":
            # 从识别的类别中筛选示例
            category_examples = [ex for ex in self.examples if ex.get("category") == category]
            if len(category_examples) >= 2:
                selected_examples = np.random.choice(category_examples, size=2, replace=False)
            else:
                # 如果该类别的示例不足两个，则使用所有可用的示例
                selected_examples = category_examples
        else:
            # 如果无法识别类别，则随机选择两个示例
            selected_examples = np.random.choice(self.examples, size=2, replace=False)

        # 将 NumPy 的 ndarray 转换为列表
        return selected_examples.tolist()

# 示例列表，包含类别标签
examples = [
    {"input": "iPhone 最新款", "output": "功能更强大的 Iphone 手机", "category": "手机"},
    {"input": "MacBook Pro", "output": "性价比更高的 苹果 笔记本", "category": "电脑"},
    {"input": "时尚衬衫", "output": "舒适的休闲裤", "category": "服装"},
    {"input": "华为 Mate 系列", "output": "遥遥领先的手机", "category": "手机"},
    {"input": "ThinkPad 笔记本", "output": "轻薄便携的办公电脑 ", "category": "电脑"},
    {"input": "夏季连衣裙", "output": "经典款牛仔裤", "category": "服装"},
    {"input": "华硕台式机", "output": "高性能的自组装电脑", "category": "电脑"},
    {"input": "运动服饰", "output": "休闲时尚的外套", "category": "服装"},
    {"input": "OnePlus 手机", "output": "高端的 Google Pixel", "category": "手机"},
    {"input": "Apple MacBook 笔记本", "output": "高端的 Apple MacBook，无敌的设计", "category": "电脑"},
]

# 实例化 CustomExampleSelector
example_selector = CustomExampleSelector(examples, CATEGORIES)

# 添加新的示例
example_selector.add_example({"input": "小米手机", "output": "性价比无敌 ", "category": "手机"})
print("更新后的示例列表:")
for ex in example_selector.examples:
    print(ex)

# 定义示例格式模板
example_formatter_template = """输入: {input} 输出: {output}\\n"""

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template=example_formatter_template,
)

# 创建 FewShotPromptTemplate
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="根据以下示例，为输入生成一个吸引人的宣传词。",
    suffix="输入: {input}\\n输出:",
    input_variables=["input"],
)

# 格式化提示
formatted_prompt = similar_prompt.format(input="iPhone 14 Pro")
print("\\n格式化后的提示:\\n", formatted_prompt)

# 初始化聊天模型
load_dotenv()

chat = ChatTongyi(
    model='qwen-plus',
    top_p=0.9,
    temperature=0.9,
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

# 调用模型并输出结果
response = chat.invoke(formatted_prompt)
print("\\n模型响应:\\n", response.content)


更新后的示例列表:
{'input': 'iPhone 最新款', 'output': '功能更强大的 Iphone 手机', 'category': '手机'}
{'input': 'MacBook Pro', 'output': '性价比更高的 苹果 笔记本', 'category': '电脑'}
{'input': '时尚衬衫', 'output': '舒适的休闲裤', 'category': '服装'}
{'input': '华为 Mate 系列', 'output': '遥遥领先的手机', 'category': '手机'}
{'input': 'ThinkPad 笔记本', 'output': '轻薄便携的办公电脑 ', 'category': '电脑'}
{'input': '夏季连衣裙', 'output': '经典款牛仔裤', 'category': '服装'}
{'input': '华硕台式机', 'output': '高性能的自组装电脑', 'category': '电脑'}
{'input': '运动服饰', 'output': '休闲时尚的外套', 'category': '服装'}
{'input': 'OnePlus 手机', 'output': '高端的 Google Pixel', 'category': '手机'}
{'input': 'Apple MacBook 笔记本', 'output': '高端的 Apple MacBook，无敌的设计', 'category': '电脑'}
{'input': '小米手机', 'output': '性价比无敌 ', 'category': '手机'}
识别的类别: 手机
\n格式化后的提示:\n 根据以下示例，为输入生成一个吸引人的宣传词。

输入: 小米手机 输出: 性价比无敌 \n

输入: iPhone 最新款 输出: 功能更强大的 Iphone 手机\n

输入: iPhone 14 Pro\n输出:
\n模型响应:\n 超凡体验，尽在 iPhone 14 Pro！
